In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
import edurel.utils.prompt_man as pm
import edurel.utils.conversation as conv
import edurel.utils.duckdb as ddb
import edurel.utils.er_agent as era


load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
ER_DIR = f"{BASE_DIR}/er"
CONFIG_DIR = f"{BASE_DIR}/config"
LOG_DIR = f"{BASE_DIR}/logs/er"


/home/iclassen/.py1/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


# Database

In [3]:
# db = ddb.Db.mem("company_en")
# db.config_exe(f"{CONFIG_DIR}/databases/company_en.yaml")

# Prompts

In [28]:
pman = pm.PromptMan(f"{ER_DIR}/building.txt")

In [30]:
eragent = era.ERAgent(
    conv.Client.STATS, 
    conv.LLMName.OPUS45,
    domain="building",
    log_dir=LOG_DIR,
)

# Testing

In [25]:
def run_er(pman, entry_id, autolog=True):
    eragent.prune_conversation(1)
    eragent.text2er(pman[entry_id][0], pman[entry_id][1], autolog=autolog)

In [ ]:
run_er(pman, 2)

In [32]:
for i in range(len(pman)):
    run_er(pman, i)